In [1]:
import os
from dotenv import load_dotenv
from semantic_kernel import Kernel

# Load environment variables
load_dotenv()

# Initialize the kernel
kernel = Kernel()

In [2]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

kernel.add_service(
    AzureChatCompletion(
        service_id="legal_advisor",
        deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    )
)

In [ ]:
legal_plugin = kernel.add_plugin(
    plugin_name="LegalPlugin",
    parent_directory="plugins"  
)

In [4]:
from semantic_kernel.functions.kernel_arguments import KernelArguments

arguments = KernelArguments(query="what could be the legal actions against a company that is not paying its employees?")
response = await kernel.invoke(
    legal_plugin["legal_advisor"],  # Must match subfolder name
    arguments=arguments
)

print(response)

**Legal Advice: Company Not Paying Its Employees**

Failure to pay wages to employees is a serious offense in India and can attract legal action under various laws. Here’s a breakdown of legal actions that may be taken:

---

### **1. Applicable Law**
- **The Payment of Wages Act, 1936**  
- **The Minimum Wages Act, 1948**  
- **Indian Penal Code (IPC)** – Section 406 (Criminal Breach of Trust) and Section 420 (Cheating)  
- **Industrial Disputes Act, 1947**  
- **Code of Wages, 2019**  

---

### **2. Summary**
1. **Payment of Wages Act, 1936**:  
   This law requires employers to pay wages on time and without unauthorized deductions. If the company fails to release salaries, employees can file a complaint with the local labor authority.

2. **Minimum Wages Act, 1948**:  
   Employers are mandated to pay at least the statutory minimum wages. Failure to do so is a punishable offense.

3. **Code of Wages, 2019**:  
   This unified law consolidates provisions for timely payment of wages,

In [5]:
import googlemaps
from typing import List, Dict
import asyncio
from concurrent.futures import ThreadPoolExecutor

class LegalServicesLocator:
    def __init__(self):
        self.gmaps = googlemaps.Client(key=os.getenv("GOOGLE_MAPS_API_KEY"))
        self.executor = ThreadPoolExecutor(max_workers=5)

    async def get_lawyer_recommendations(self, location: str, practice_area: str = "lawyer") -> List[Dict]:
        """Get top 5 lawyers near location with enhanced error handling"""
        try:
            loop = asyncio.get_running_loop()
            places = await loop.run_in_executor(
                self.executor,
                lambda: self.gmaps.places_nearby(
                    location=location,
                    keyword=practice_area,
                    rank_by="distance",
                    type="lawyer",
                    language="en"
                )
            )
            
            # Enhanced sorting with fallbacks
            return sorted(
                places.get("results", []),
                key=lambda x: (
                    -x.get("rating", 0),
                    x.get("user_ratings_total", 0),
                    x.get("price_level", 4)
                )
            )[:5]
            
        except Exception as e:
            print(f"Google Maps API error: {str(e)}")
            return []

    def format_results(self, lawyers: List[Dict]) -> str:
        """Format results for LLM response"""
        if not lawyers:
            return "No lawyers found in your area."
            
        return "\n".join(
            f"{idx+1}. {lawyer['name']} ⭐ {lawyer.get('rating', 'N/A')}\n"
            f"   📍 {lawyer['vicinity']}\n"
            f"   📞 {lawyer.get('international_phone_number', 'Contact not available')}\n"
            for idx, lawyer in enumerate(lawyers)
        )

In [6]:
# Initialize services
locator = LegalServicesLocator()
legal_tools = kernel.add_plugin(plugin_name="LegalPlugin", parent_directory="plugins")

async def handle_legal_request(user_query: str, user_location: str):
    """End-to-end legal assistance flow"""
    # Step 1: Get legal advice
    advice = await kernel.invoke(
        legal_tools["legal_advisor"],
        arguments=KernelArguments(query=user_query)
    )
    
    # Step 2: Check if lawyer needed
    needs_lawyer = await kernel.invoke(
        legal_tools["needs_lawyer_decider"],
        arguments=KernelArguments(
            legal_advice=str(advice),
            user_query=user_query
        )
    )
    
    # Step 3: Get recommendations if needed
    if "YES" in str(needs_lawyer).upper():
        lawyers = await locator.get_lawyer_recommendations(user_location)
        return {
            "advice": str(advice),
            "recommendations": locator.format_results(lawyers)
        }
    return {"advice": str(advice)}

In [7]:
response = await handle_legal_request(
    user_query="I need help with a divorce case",
    user_location="40.7128,-74.0060"  # NYC coordinates
)
print(response["advice"])
print(response["recommendations"])

Certainly, I’m here to help you with your divorce case. Here's a breakdown:

---

### 1. **Applicable Law**:
   The laws applicable for divorce in India depend on your religion and the governing personal law. Below are the key legislations:
   - **Hindu Marriage Act, 1955** (for Hindus, including Sikhs, Buddhists, and Jains).
   - **Special Marriage Act, 1954** (for interfaith or secular marriages).
   - **Indian Divorce Act, 1869** (for Christians).
   - **Muslim Personal Law (Shariat) Application Act, 1937** (for Muslims).
   - **Parsi Marriage and Divorce Act, 1936** (for Parsis).
   Additionally:
   - **Code of Civil Procedure, 1908 (CPC)** governs procedure for divorce petitions.
   - **Indian Penal Code, 1860 (IPC)** applies if there are claims of cruelty, adultery, or desertion.
  
---

### 2. **Summary**:
   1. **Types of Divorce**:
      - **Mutual Consent Divorce**: Both parties agree to the divorce under the applicable personal law.
      - **Contested Divorce**: Filed by on